# Optimizers

This notebook serves as a library of optimizers, as well as a set of test functions that evaluate optimizer performance



In [1]:
#check if the notebook is being used as a library, or being run as a notebook
asLibrary = lambda : '__file__' in globals()
if asLibrary():
    print("optimizers run as library")

## Define the the optimizer interface
to maitain a level of decoupling between elements of the software system, the optimizer will not be given access to the robot or the experiment explicitly, but will deal instead only with function handles for the objective function and constraint functions.

In [2]:
#define optimization interface here
class Opt:
    """
    defines the interface that all types of optimizer should follow
    """
    def optimize(self,obj,bounds,constraints):
        """
        the function which actually performs the optimization or sampling
        inputs:
            obj - a function handle from exp.objective
            bounds - an [2xn] np.array of box constraints
            constraints - a list of function handles to hard constraint functions (to be implemented)
        outputs:
            x - a list of np.array objects, each of which is a solution
                list of the parameter for the optimal solution
        """
        pass


to do: 
* write a BFGS class, and try to use it to solve for an optimal solution, within the specified bounds. 




* extend this to include solving the problem n times, from different sobol sampled locations. 
* view the solutions
* work on sobol sampling + dispersion optimization.
* if we have a good design, discuss what needs to be improved on the human end to make this an actually good model, including some of the other features you wanted. 
* start working on penalty functions, or on 

from future import:
* might be nice at some point to develop a visualizer to visualize different trajectories on the robot with an animate button



In [133]:
from scipy.interpolate import interp1d
import plotly.graph_objs as go
import numpy as np

# objective function
class testObjFxn:
    def __init__(self):
        self.X = np.linspace(0,1,1000)
        self._y = np.asarray([self.objective(x) for x in self.X])
        self.f =  interp1d(self.X, self._y,kind='cubic')
        
    def objective(self,x, noise=0.1):
        noise = np.random.normal(loc=0, scale=noise)
        return (x**2 * np.sin(5 * np.pi * x)**6.0) + noise
    
    def pltSampleHistory(self,x,fx):
        n = len(x)
        
        #x vs. f(x)
        fig = go.Figure()
        fig.add_scatter(mode="markers", marker = dict(color = np.linspace(0,1,n)),name="Sample")
        fig.data[0]['x'] = x
        fig.data[0]['y'] = fx
        display(fig)
        
        
        #n vs fx
        fig2 = go.Figure()
        fig2.add_scatter(x=np.arange(0,n,1),y=fx,name="Sample")
        display(fig2)
        
        

### baysian optimization

#### resources
* https://gpflowopt.readthedocs.io/en/latest/notebooks/constrained_bo.html baysian optimization with constraints.



In [ ]:
# # example of bayesian optimization for a 1d function from scratch
# from math import sin
# from math import pi
# from numpy import arange
# from numpy import vstack
# from numpy import argmax
# from numpy import asarray
# from numpy.random import normal
# from numpy.random import random
# from scipy.stats import norm
# from sklearn.gaussian_process import GaussianProcessRegressor
# from warnings import catch_warnings
# from warnings import simplefilter
# from matplotlib import pyplot

# # objective function
# def objective(x, noise=0.1):
#     noise = normal(loc=0, scale=noise)
#     return (x**2 * sin(5 * pi * x)**6.0) + noise

# # surrogate or approximation for the objective function
# def surrogate(model, X):
#     # catch any warning generated when making a prediction
#     with catch_warnings():
#         # ignore generated warnings
#         simplefilter("ignore")
#         return model.predict(X, return_std=True)

# # probability of improvement acquisition function
# def acquisition(X, Xsamples, model):
#     # calculate the best surrogate score found so far
#     yhat, _ = surrogate(model, X)
#     best = max(yhat)
#     # calculate mean and stdev via surrogate function
#     mu, std = surrogate(model, Xsamples)
#     mu = mu[:, 0]
#     # calculate the probability of improvement
#     probs = norm.cdf((mu - best) / (std+1E-9))
#     return probs

# # optimize the acquisition function
# def opt_acquisition(X, y, model):
#     # random search, generate random samples
#     Xsamples = random(100)
#     Xsamples = Xsamples.reshape(len(Xsamples), 1)
#     # calculate the acquisition function for each sample
#     scores = acquisition(X, Xsamples, model)
#     # locate the index of the largest scores
#     ix = argmax(scores)
#     return Xsamples[ix, 0]

# # plot real observations vs surrogate function
# def plot(X, y, model):
#     # scatter plot of inputs and real objective function
#     pyplot.scatter(X, y)
#     # line plot of surrogate function across domain
#     Xsamples = asarray(arange(0, 1, 0.001))
#     Xsamples = Xsamples.reshape(len(Xsamples), 1)
#     ysamples, _ = surrogate(model, Xsamples)
#     pyplot.plot(Xsamples, ysamples)
#     # show the plot
#     pyplot.show()
    

# if not asLibrary():
#     # sample the domain sparsely with noise
#     X = random(100)
#     y = asarray([objective(x) for x in X])
#     # reshape into rows and cols
#     X = X.reshape(len(X), 1)
#     y = y.reshape(len(y), 1)
#     # define the model
#     model = GaussianProcessRegressor()
#     # fit the model
#     model.fit(X, y)
#     # plot before hand
#     plot(X, y, model)
#     # perform the optimization process
#     for i in range(100):
#         # select the next point to sample
#         x = opt_acquisition(X, y, model)
#         # sample the point
#         actual = objective(x)
#         # summarize the finding
#         est, _ = surrogate(model, [[x]])
#         print('>x=%.3f, f()=%3f, actual=%.3f' % (x, est, actual))
#         # add the data to the dataset
#         X = vstack((X, [[x]]))
#         y = vstack((y, [[actual]]))
#         # update the model
#         model.fit(X, y)

#     # plot all samples and the final surrogate function
#     plot(X, y, model)
#     # best result
#     ix = argmax(y)
#     print('Best Result: x=%.3f, y=%.3f' % (X[ix], y[ix]))

### sobol sampling

the idea behind this approach is to evaluate:
1. if the function we are dealing with is convex, (in which case, all of the simplex methods will end up at the same place)
2. if it's not convex, are there local solutions that are interesting to consider (perhaps are less sensitive, or better in some way we didn't anticipate?) the idea would be to visually inspect these solutions for their quality.


#### refs:
https://people.sc.fsu.edu/~jburkardt/py_src/sobol/sobol.html

In [148]:
from sobol import i4_sobol
import ipywidgets as widgets
from IPython.display import display
import plotly.graph_objs as go
import numpy as np

    
class Sobol:
    """
    the sobol opt class 
    """
    def __init__(self):
        self.n = 1000          #number of samples to evaluate
        self.nBest = 10        #number to keep
        self.xBest = []        #list of nBest x values
        self.fxBest = []       #list of nBest obj function values
        self.randomSeed = True #should sample seed be randomized?
        
        self.selectionMode = "max_fx" #str name of selection fxn ["max_fx", "obj_var_tradeoff"]
        self.W_obj = 1                #importance parameter for objective fxn
        self.W_dispersion = 1         #importance parameter for dispersion metric 
        
    
    @staticmethod
    def twoPointLine(x,x1,y1,x2,y2):
        """returns y given the formula of a line specified in 2 point form"""
        return ((y2-y1)/(x2-x1))*(x - x1) + y1
    
    @staticmethod
    def Sample(bounds,n,randomSeed = True):
        """
        inputs:
            bounds - [nx2] array of [max,min]
            n - number of sample points requested
            randomSeed - bool - start in a place other than 0
        outputs:
            samps - list of np.arrays  (so you can interate through them more easily)
        """
        #calc the sobol inputs
        ndims = bounds.shape[0]
        upperbounds = bounds[:,0] ; lowerbounds = bounds[:,1]
        if randomSeed: seed_0 = np.random.randint(0,10000)
        else: seed_0 = 0
        
        #sample the unit hypercube
        samps = [i4_sobol(ndims, seed)[0] for seed in range(seed_0,n + seed_0)]
        #samps = np.array(samps)
        
        #shift and scale from the unit hypercube to the bounds rectangular prism
        sl = []  #samps list
        for sample in samps:
            sn = np.zeros(ndims)  #sample new
            for i,x in enumerate(sample):
                x1 = 0 ; x2 = 1
                y1 = lowerbounds[i] ; y2 = upperbounds[i]
                sn[i] = Sobol.twoPointLine(x,x1,y1,x2,y2)
            sl.append(sn)
        
        return sl
    
    def max_fx(self,xs,fxs):
        """
        return the nBest results based only on the value of Fx
        """
        np.argsort()
        
    
    def optimize(self,obj,bounds,constraints):
        """
       discover interesting and good solutions at different locations within the space, 
       no optimization is performed strictly speaking, just structured exploration
        """
        xs = Sobol.Sample(bounds,self.n,self.randomSeed)
        fxs = [] 
        for x in xs:
            fx.append(obj(x))
        
        return eval('self.' + self.selectionFxn + '(xs,fxs)')


class SobolVis:
    """
    just want to get a sense for what sampling a unit space
    using sobol sampling looks like
    """
    def __init__(self):
        _fig = go.Figure(data=[go.Scatter3d(z=[0],x=[0],y=[0], name = "sampling",mode="markers",
                                            marker = dict(size=10,color = np.linspace(0,1,1000),showscale=True))])
        self.fig = go.FigureWidget(_fig)
        n = widgets.IntSlider(min=1,max=1000,value=1,description='samps')
        self.wdict = {"n":n}
        
        self.fig.update_layout(
            autosize=False,
            width = 800,
            height= 800,
            legend=dict(x=.025, y=.975),
            margin=dict(l=0, r=20, t=0, b=0))
        
    def genSamps(self,n):
        #how to use sobol
        bounds = np.array([[.5,1,-1],[.25,-1,-3]]).T
        samps = Sobol.Sample(bounds,n,randomSeed = False)
        #samps = [i4_sobol(dim_num, seed)[0] for seed in range(n)]
        samps = np.array(samps)
        xs = samps[:,0] ; ys = samps[:,1] ; zs = samps[:,2]
        return xs,ys,zs
    
    
    def update(self,n=1):
        fig = self.fig
        with fig.batch_update():
            #plot n sobol points, with coloring
            xs,ys,zs = self.genSamps(n)
            
            fig.data[0]['x']= xs
            fig.data[0]['y']= ys
            fig.data[0]['z']= zs
            
            
    def disp(self):
        self.out = widgets.interactive_output(self.update, self.wdict)
        display(self.wdict["n"])
        display(self.fig)
        

#interactive 3D sobol sampling visualization to get a sense of the algorithm 
if not asLibrary() and True:
    vis = SobolVis()
    vis.disp()

IntSlider(value=1, description='samps', max=1000, min=1)

FigureWidget({
    'data': [{'marker': {'color': array([0.      , 0.001001, 0.002002, ..., 0.997998, 0.998999,…

### BFGS 
Broyden–Fletcher–Goldfarb–Shanno (BFGS) is a quazi-newton optimization method that uses the secant approach to find stationary points via locating the zeros of the gradient (jacobian) function. 

In [168]:
from scipy.optimize import minimize
from scipy.optimize import Bounds

class BFGS:
    def __init__(self):
        self.mode = "seeded"   #["seeded","sobol","SA"] 
        self.n = 1             #number of starting points
        self.x0 = []           #incase you want to prime the algorithm, from outside manipulation
        self.xBest = []        #list of n x values
        self.fxBest = []       #list of n obj function values
        self.res = []          #results of most recent optimization
        
    def optimize(self,obj,bounds,constraints): #refactor to remove this code repetition.
        
        #specify the bounds:
        bnds = Bounds(bounds[:,1],bounds[:,0],keep_feasible=True)
        
        #specify the fixed step size (epsilon):
        eps = (bounds[:,0] - bounds[:,1])/10000
        
        #make a maximum into a minimum
        min_obj = lambda x: -1 * obj(x)
        
        if self.mode == "seeded":
            self.res = minimize(min_obj,
                                self.x0[0],
                                method='BFGS',
                                jac = False,
                                bounds = bnds,
                                options={'gtol': 1e-3,
                                         'disp': True,
                                         'eps': eps})
                                        #'finite_diff_rel_step':finite_diff_rel_step})
            self.xBest.append(self.res.x)
            self.fxBest.append(1)
            
        elif self.mode == "sobol":
            samps = Sobol.Sample(bounds,self.n,randomSeed = True)
            for x0 in samps:
                self.res = minimize(obj, x0, method='BFGS',
                           options={'gtol': 1e-6, 'disp': True})
                self.xBest.append(self.res.x)
                self.fxBest.append(1)
        
        return self.xBest

### Simulated Annealing

In [134]:
#simulated anealing algorithm must have the following functions: 
# the feasable state space
# the Energetic function (which is the objective function) E()
# the candidate generator procedure (neighbor(x))
#        -generation of random candidate vector, with variable magnitude?
#         aniostropic exploration is better!!!
#thoughts for how I might solve this problem - 
#         boundry check or only sampling within the boundry seems a good strategy. 
#        -moving in one direction at at time. 

# acceptance probability function P()
# anealing schedule - number, number of anealing cycles, cooling rate
# some termination condition - total execution time or number of interations
# presensce of and number of restarts, deterministic or otherwise? 

#of these, cooling schedule and neighbor seem the most important functions


class SA:
    def __init__(self):
        self.nIter = 1000;                     #total number of iterations to perform
        self.nResets = 3                       #number of resets to best 
        self.brt = self._brt()                 #trial numbers during which to reset to best
        self.dist = "uniform"                  #neighborhood sampling method 
        self.aMode = "exp"                     #Annealing mode ["exp","linear","stepped"]
        self.nMode = "linear"                  #neighborhood Mode
        self.T = 1                             #temperature variable on [1-0)
        self.N = 1                             #neighborhood size coefficient [1-0)
        self.C_half = 200                      #half-life cooling              [trials]
        self.N_half = 150                      #half-life of neighborhood size [trials]
        self.fxBest = 0                        #best encountered function value
        self.xBest = []                        #location of best value
        self.dbg = False                        #save variables for debugging
        self.xCache  = []                      #storage for x
        self.FxCache = []                      #storage for Fx
        self.updateReports = True              #should we print out an update of how the optimization is progressing?
        self.updateCount = 10                  #number of updates
        
        
    def setBounds(self,bcs):
        self.x_max = bcs[:,0]                  #upper limits on box constraints for x variable
        self.x_min = bcs[:,1]                  #lower limits on box constraints for x variable
        
    def _center(self,x_max,x_min):
        """
        find  the center of a box defined by limits
        """
        return [(x_max[i] + x_min[i])/2 for i in range(len(x_max))]
    
    def _twoPointLine(self,x,x1,y1,x2,y2):
        """
        returns y given the formula of a line specified in 2 point form
        """
        return ((y2-y1)/(x2-x1))*(x - x1) + y1
    
    def _brt(self):
        """
        calculate the best reset trials, evenly distribute in n_iter
        """
        return np.random.randint(low  = int(self.nIter/5),
                                 high = self.nIter,
                                 size = self.nResets)
        
            
    def sample(self,x):
        """
        sample the space for the next sample based on the neighborhood
        function
        input:
            x
        output:
            x_new
        """
        #define the (ever-shrinking) neighborhood bounding box
        w = self.N * ((self.x_max - self.x_min)/2) 
        nx_max = x + w
        nx_min = x - w
        
        #define the combined bounding box as an intersection between
        #the box constraints and the neighborhood box - a mini-max problem
        cx_max = np.amin(np.vstack((self.x_max,nx_max)),axis = 0)
        cx_min =  np.max(np.vstack((self.x_min,nx_min)),axis = 0)
        
#         for i in range(len(self.x_max)):
#             cx_max[i] = min(self.x_max,nx_max[i])
#             cx_min[i] = max(self.x_min,nx_min[i])
        
        #sample from the combined box, according to the appropriate distribution
        if self.dist == "uniform": 
            x_new = np.random.uniform(cx_min,cx_max) #vectorized
            
        if self.dist == "normal":
            pass
#             zScore = 1.645 #95% confidence interval
#             x_new = []
#             while len(x_new) > len(self.x_min):
#                 x = np.random.normal(loc=0.0, scale=1.0, size=None)
#                 if abs(x) < zScore:
#                     x_new.append(x)
#             c_range = cx_max - cx_min
            
 
        return x_new

    def optimize(self,obj,bounds,constraints):
        
        #set box constraints
        bcs = bounds()
        self.setBounds(bcs)
        
        #init x0 and fx0
        x0 =  self._center(self.x_max,self.x_min)
        fx0 = obj(x0)
        
        for n in range(self.nIter + 1):
            x = self.sample(x0)
            fx = obj(x)
            
            #if better, always keep
            if fx > fx0:
                x0 = x ; fx0 = fx
                
            #probabalistically accept worse answer
            else:
                Δf = fx - fx0
                r = np.random.random()
                if r < np.exp(Δf/self.T):
                    fx0 = fx ; x0 = x
                    #print(x)
                else:
                    pass
             
            
            #maintain best (for restarts, if used)
            if fx > self.fxBest:
                self.xBest = x  ; self.fxBest = fx 
                
            #do a best restart:
            if n in self.brt:
                  x0 = self.xBest ; fx0 = self.fxBest 
            
            #decrease temperature and neighborhood size
            if self.aMode == "exp": self.T = .5**(n/self.C_half)
            if self.nMode == "exp": self.N = .5**(n/self.N_half)
            
            if self.aMode == "Linear": self.T = self._twoPointLine(n,0,1,self.nIter,0)
            if self.nMode == "Linear": self.N = self._twoPointLine(n,0,1,self.nIter,0)
                
                  
            #store debugging vars
            if self.dbg:
                self.xCache.append(x0[0])
                self.FxCache.append(fx0[0])
                
            #give updates
            if self.updateReports:
                if n % ((self.nIter) / 10) == 0:
                    print("on iteration %d" %n)
        
        return x0
                    
 





# if not asLibrary():
#     #plot
#     tester = testObjFxn()
#     boxConstraints = np.array([[1,0]])
#     sa = SA()
#     sa.dbg = True
#     sa.setBounds(boxConstraints)

#     x0,fx0 = sa.optimize(tester.f)
#     print(x0,fx0)
#     print(sa.xBest)
    
#     tester.pltSampleHistory(sa.xCache,sa.FxCache)
    
#     #%timeit sa.optimize(tester.f)

TypeError: optimize() missing 2 required positional arguments: 'bounds' and 'constraints'

# notes:

* instead of optimization per se, you could try something where you take the best solutions from a variety of locations within the space, so do a giant sobol sample on the space, and then choose the 100 best solutions that are the most different in terms of euclidian distance within the space, something like that. 
* I think we might want to add the ability to save general information within the picking process. so we could save an experiment like the one above. 